Gerar dataset sintético com regras de fraude

In [5]:
# Cell 1: imports e definições
import pandas as pd
import random

ufs = ['SP','RJ','MG','PR','RS','BA']
tipos_conta = ['corrente','poupanca','salario']
chaves_pix = [
    '99638194710','user2505@email.com',
    '55522120872','+5511941273847','86281025445'
]
# histórica de fraude: True/False
historico_fraude = [True, False]

def gerar_registro():
    valor = round(random.uniform(10, 3000), 2)
    hora  = random.randint(0, 23)
    local = random.choice(ufs)
    tipo  = random.choice(tipos_conta)
    chave = random.choice(chaves_pix)
    hist  = random.choice(historico_fraude)

    # regras Versão 1
    if hist:
        fraude = 1
    elif 0 <= hora <= 6 and valor > 300:
        fraude = 1
    elif 7 <= hora <= 23 and valor > 2000:
        fraude = 1
    else:
        fraude = 0

    return {
        'valor': valor,
        'hora': hora,
        'localizacao': local,
        'tipo_conta': tipo,
        'chave_pix': chave,
        'chave_historico_fraude': hist,
        'fraude': fraude
    }

# gera N registros e salva
N = 1000
df = pd.DataFrame([gerar_registro() for _ in range(N)])
df.to_csv('dados_pix_sinteticos.csv', index=False)
print("✅ dados_pix_sinteticos.csv criado com sucesso!")
print(df.head())


✅ dados_pix_sinteticos.csv criado com sucesso!
     valor  hora localizacao tipo_conta           chave_pix  \
0  2861.60    11          MG    salario         55522120872   
1  1018.76     3          RS   poupanca  user2505@email.com   
2  1769.76    22          SP   corrente         86281025445   
3  1868.14    20          SP   corrente  user2505@email.com   
4  1321.03    12          BA    salario  user2505@email.com   

   chave_historico_fraude  fraude  
0                    True       1  
1                   False       1  
2                   False       0  
3                    True       1  
4                   False       0  


Extrair CSV

In [6]:
from google.colab import files
files.download('dados_pix_sinteticos.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Dicionário de colunas


In [ ]:
column_descriptions = {
    'valor': 'Valor da transação em reais',
    'hora': 'Hora da transação (0–23)',
    'localizacao': 'UF de origem da transação',
    'tipo_conta': 'Tipo de conta (corrente, poupança, salário)',
    'chave_pix': 'Identificador da chave PIX (CPF, e-mail, etc.)',
    'chave_historico_fraude': 'Se a chave já foi identificada em fraude (True/False)',
    'cliente_tem_historico_madrugada': 'Se o cliente já fez PIX na madrugada antes (True/False)',
    'flag_madrugada_incomum': 'Se a transação ocorreu entre 0h e 6h (True/False)',
    'flag_fraude_comportamental': 'Flag combinada de comportamento suspeito (True/False)',
    'fraude': 'Rótulo alvo: 1=fraude, 0=legítima'
}

Pipeline de treinamento (passo a passo)
1. Importar bibliotecas




In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pickle

2. Carregar os dados

In [8]:
df = pd.read_csv("dados_pix_sinteticos.csv")

3. Pré-processamento

In [9]:
# 2) Pré-processamento: label encode
le_loc = LabelEncoder()
df['localizacao'] = le_loc.fit_transform(df['localizacao'])
le_tipo = LabelEncoder()
df['tipo_conta'] = le_tipo.fit_transform(df['tipo_conta'])

4. Dividir variáveis preditoras e alvo

In [10]:
# 3) Features e target
X = df.drop(columns=['fraude','chave_pix'])
y = df['fraude']

5. Separar em treino e teste

In [11]:
# 4) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

6. Treinar o Random Forest

In [12]:
# 5) Treino
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

7. Avaliar desempenho

In [13]:
# 6) Avaliação
y_pred = modelo.predict(X_test)
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de classificação:\n", classification_report(y_test, y_pred))


Matriz de confusão:
 [[ 43   0]
 [  0 157]]

Relatório de classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00       157

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



8. Importância das variáveis

In [14]:
# 7) Importâncias
importancias = pd.Series(modelo.feature_importances_, index=X.columns)
print("\nImportância das features:\n", importancias.sort_values(ascending=False))



Importância das features:
 chave_historico_fraude    0.414312
valor                     0.352963
hora                      0.216254
localizacao               0.011443
tipo_conta                0.005028
dtype: float64


In [15]:
# 8) Salvar o modelo
with open('modelo_random_forest_pix.pkl','wb') as f:
    pickle.dump(modelo, f)
print("✅ modelo_random_forest_pix.pkl salvo com sucesso!")

✅ modelo_random_forest_pix.pkl salvo com sucesso!


In [16]:
from google.colab import files
files.download('modelo_random_forest_pix.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>